In [26]:
# Note to self - to install a package globally, pip install it,
# then run jupyter notebook again

'''
Kelly Criterion
Expected Value of Game = (Probability(Win) - Probability(Loss)) * Size of Bet > 0

The available capital after n rounds of game is:
c(n) = c(0) * (1+f)^h - (1+f)^t
where
c(n) = capital at round n
h = probability of win
t = probability of loss 
f = fraction of capital to commit

So the closed form log-return maximization problem becomes:
r(g) = (h/n) * log(1+f) + (t/n) * log(1+f)
i.e. the sum of the product winning probability divided by the number of trials and the product of 
the losing probability divided by the number of trials

So the f term is the maximization of the expected value of the rate of growth, which is the function G(f)
E[h] = n * p
E[t] = n * q

Then 
E[r^g] = p * log(1+f) + q * log(1+f)
= G(f)

Taking the first derivative of G(f):
G'(f) = (p/1+f) - (q/1+f)
= (p - pf - q - qf)/((1+f)*(1-f))
= (p - q - f)/ ((1+f)*(1-f))
or the quotient of (trails * win fraction) - (trails * loss fraction) - (fraction) and the product (1 + fraction)
and (1 - fraction)

so finally the optimal fraction is:
f(n) = p - q
REAL NUMBER TIME!
If win chance = 55 &
lose change = 45
55 - 45 = 10, so invest 10% of capital per bet,

of course, constraining for max position size in any given security

Let's do it in code:
'''

"\nKelly Criterion\nExpected Value of Game = (Probability(Win) - Probability(Loss)) * Size of Bet > 0\n\nThe available capital after n rounds of game is:\nc(n) = c(0) * (1+f)^h - (1+f)^t\nwhere\nc(n) = capital at round n\nh = probability of win\nt = probability of loss \nf = fraction of capital to commit\n\nSo the closed form log-return maximization problem becomes:\nr(g) = (h/n) * log(1+f) + (t/n) * log(1+f)\ni.e. the sum of the product winning probability divided by the number of trials and the product of \nthe losing probability divided by the number of trials\n\nSo the f term is the maximization of the expected value of the rate of growth, which is the function G(f)\nE[h] = n * p\nE[t] = n * q\n\nThen \nE[r^g] = p * log(1+f) + q * log(1+f)\n= G(f)\n\nTaking the first derivative of G(f):\nG'(f) = (p/1+f) - (q/1+f)\n= (p - pf - q - qf)/((1+f)*(1-f))\n= (p - q - f)/ ((1+f)*(1-f))\nor the quotient of (trails * win fraction) - (trails * loss fraction) - (fraction) and the product (1 + f

In [27]:
import math
import time
import numpy as np
import pandas as pd
import datetime as dt
import cufflinks as cf
from pylab import plt

ModuleNotFoundError: No module named 'cufflinks'

In [28]:
np.random.seed(1000)
plt.style.use('seaborn')
%matplotlib inline

NameError: name 'plt' is not defined

In [29]:
# Monte Carlo time!
p = 0.55
f = p  - (1 - p) # has to add to 1

In [30]:
f

0.10000000000000009

In [11]:
I = 50 # trials
n = 50 # bets per trial

In [12]:
def run_simulation(f):
    c = np.zeros((n, I)) # zeros matrix surface n = rows I = columns
    c[0] = 100 # initial capital is par
    for i in range(I):
        for t in range(1, n):
            o = np.random.binomial(1, p) # digitize the wins/losses against the probability of win
            if o > 0:
                c[t, i] = (1 + f) * c[t-1, i] # winrar
            else:
                c[t, i] = (1-f) * c[t-1, i] # loss
    return c

In [13]:
c_1 = run_simulation(f)

In [14]:
c_1.round(2)

array([[100.  , 100.  , 100.  , ..., 100.  , 100.  , 100.  ],
       [ 90.  , 110.  , 110.  , ..., 110.  , 110.  , 110.  ],
       [ 99.  ,  99.  , 121.  , ..., 121.  , 121.  ,  99.  ],
       ...,
       [106.7 , 194.81, 130.41, ..., 106.7 ,  47.81,  87.3 ],
       [117.37, 175.33, 143.45, ...,  96.03,  52.59,  78.57],
       [129.1 , 192.86, 129.1 , ...,  86.42,  57.85,  86.42]])

In [15]:
plt.figure(figsize=(20,12))
plt.plot(c_1, 'b', lw=0.5)
plt.plot(c_1.mean(axis=1), 'r', lw=2.5);

NameError: name 'plt' is not defined

In [16]:
c_2 = run_simulation(0.05)

In [17]:
c_3 = run_simulation(0.25)

In [18]:
c_4 = run_simulation(0.5)

In [19]:
plt.figure(figsize=(20,12))


NameError: name 'plt' is not defined

In [ ]:
'''
Dot Products: for element A[0][c], sum up A[0][c] * B[r][0] for r in A
i.e. hold the row constant for matrix 1, hold the column constant for matrix 2
'''
a = np.array([[1,2,3],[4,5,6]])
b = np.array([[1,2,3],[3,4,5],[5,6,7]])